<a href="https://colab.research.google.com/github/Mehak-shahani/my-projects-/blob/main/Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers
!pip install sentence-transformers
!pip install vaderSentiment


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 2.9 MB/s eta 0:00:00


In [2]:
# Import necessary libraries
from transformers import pipeline
from sentence_transformers import SentenceTransformer, util
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [3]:
# Initialize the models
faq_model = SentenceTransformer('all-MiniLM-L6-v2')  # For FAQ matching
sentiment_analyzer = SentimentIntensityAnalyzer()  # For sentiment analysis
question_answering = pipeline("question-answering")  # Hugging Face QA pipeline

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/261M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

Device set to use cpu


In [4]:
# Predefined FAQ dataset
faq_data = [
    {"question": "What are your business hours?", "answer": "We are open from 9 AM to 5 PM, Monday to Friday."},
    {"question": "What services do you provide?", "answer": "We provide AI solutions for businesses, including chatbots and task automation."},
    {"question": "How can I contact support?", "answer": "You can email us at support@example.com or call us at +123456789."},
    {"question": "What is your refund policy?", "answer": "We offer a 30-day refund policy for all purchases."}
]

In [5]:
# Extract questions from the FAQ dataset
faq_questions = [item['question'] for item in faq_data]
faq_embeddings = faq_model.encode(faq_questions, convert_to_tensor=True)  # Embed the questions

In [6]:
# Define chatbot function
def chatbot_response(user_input):
    # Step 1: Sentiment Analysis
    sentiment_score = sentiment_analyzer.polarity_scores(user_input)
    if sentiment_score['compound'] <= -0.5:
        return "I'm sorry to hear that. Let me escalate this to our support team. They will get back to you shortly!"

In [ ]:
# Compute the embedding for the user input
user_input = input("Please enter your question: ")
user_embedding = faq_model.encode(user_input, convert_to_tensor=True)
# Compute the cosine similarities between the user input and the FAQ questions
similarities = util.pytorch_cos_sim(user_embedding, faq_embeddings)[0]

# Get the index of the FAQ with the highest similarity score
best_match_idx = similarities.argmax()

# Get the best score for the matched FAQ
best_score = similarities[best_match_idx].item()
if best_score > 0.7:
    print(f"Chatbot: {faq_data[best_match_idx]['answer']}")
else:
    print("Chatbot: I'm not sure about that. Let me pass your query to our human support team.")




In [ ]:
# Test the chatbot
while True:
    user_input = input("You: ")
    if user_input.lower() == "exit":
        print("Chatbot: Goodbye!")
        break
    response = chatbot_response(user_input)
    print(f"Chatbot: {response}")